In [ ]:
!wget https://he-s3.s3.amazonaws.com/media/hackathon/hackerearth-test-draft-1-158/predict-the-match-percentage-25-818cf487/4505f0c034b511eb.zip

--2020-12-23 03:27:26--  https://he-s3.s3.amazonaws.com/media/hackathon/hackerearth-test-draft-1-158/predict-the-match-percentage-25-818cf487/4505f0c034b511eb.zip
Resolving he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)... 52.219.32.176
Connecting to he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)|52.219.32.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638534 (624K) [application/zip]
Saving to: ‘4505f0c034b511eb.zip’

4505f0c034b511eb.zi 100%[===================>] 623.57K  2.67MB/s    in 0.2s    

2020-12-23 03:27:27 (2.67 MB/s) - ‘4505f0c034b511eb.zip’ saved [638534/638534]



In [ ]:
!unzip 4505f0c034b511eb.zip

Archive:  4505f0c034b511eb.zip
   creating: dataset/
  inflating: dataset/data.csv        
  inflating: dataset/sample dataset.csv  
  inflating: dataset/sample submission.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('dataset/data.csv')
sub = pd.read_csv('dataset/sample submission.csv')

In [ ]:
df=train.copy()
train.head(5)

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,location,pets,smokes,language,new_languages,body_profile,education_level,dropped_out,bio,interests,other_interests,location_preference
0,fffe3100,Edith Lopez,27,single,f,gay,socially,never,66.0,medicine / health,"oakland, california",likes dogs and likes cats,no,"english (fluently), spanish (poorly), sign lan...",interested,athletic,4.0,no,bottom line i love life! i work hard and i lov...,sports,instruments,same state
1,fffe3200,Travis Young,26,single,m,gay,socially,never,68.0,other,"pleasant hill, california",likes dogs,no,"english (fluently), tagalog (okay), french (po...",interested,fit,3.0,no,"i'm a straightforward, genuine, fun loving (i'...",painting,instruments,anywhere
2,fffe3300,Agnes Smith,20,seeing someone,f,bisexual,socially,sometimes,69.0,other,"oakland, california",has dogs and likes cats,sometimes,"english (fluently), sign language (poorly), fr...",interested,fit,2.0,no,mmmmm yummy tacosss. yoga is where it's at. i ...,instruments,dancing,same city
3,fffe3400,Salvador Klaver,27,single,m,bisexual,socially,sometimes,68.0,computer / hardware / software,"daly city, california",likes dogs and likes cats,no,english,not interested,average,3.0,no,i'm a stealth geek. that special mix of techni...,sketching,acting,same city
4,fffe3500,Elana Sewell,22,single,f,bisexual,often,sometimes,68.0,other,"oakland, california",likes dogs and likes cats,yes,english,not interested,average,2.0,yes,with the whisper of the wind i was weaved into...,craft,designing,same city


In [ ]:
x=[]
y=[]
z=[]
for i in df['job']:
  j=i.split('/')
  z.append(len(j))
  x.append(j[0])
  if len(j)>1:
    y.append(j[1])
  else:
    y.append('other')
df['job_length']=z
df['job_f']=x
df['job_s']=y

In [ ]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

df['bio'] = df['bio'].apply(clean_text)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
punctuation=string.punctuation
df['word_count']=df['bio'].apply(lambda x: len(str(x).split(" ")))
df['char_count'] = df['bio'].str.len()
def avg_word(sentence):
    words = sentence.split()
    return ((sum(len(word) for word in words))/(len(words)+1))

df['avg_word'] = df['bio'].apply(lambda x: avg_word(x))
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['stopwords'] = df['bio'].apply(lambda x: len([x for x in x.split() if x in stop]))
#df['numerics'] = df['bio'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
#df['upper'] = df['bio'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df['word_density'] = df['char_count'] / (df['word_count']+1)
#df['punctuation_count'] = df['bio'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation)))

In [ ]:
j=[]
for i in df['bio']:
  j.append(len(i))
df['len']=j

In [ ]:
from textblob import TextBlob
df['polarity'] = df.apply(lambda x: TextBlob(x['bio']).sentiment.polarity, axis=1)
df['subjectivity'] = df.apply(lambda x: TextBlob(x['bio']).sentiment.subjectivity, axis=1)

In [ ]:
del df['bio']

In [ ]:
x=[]
y=[]
for i in df['location']:
  m=i.split(',')
  x.append(m[0])
  y.append(m[1])
df['street']=x
df['country']=y
del df['location']

In [ ]:
x=[]
y=[]
z=[]
for i in df['language']:
  j=i.split(',')
  z.append(len(j))
  x.append(j[0])
  if len(j)>1:
    y.append(j[1])
  else:
    y.append('other')
df['language_length']=z
df['language_f']=x
df['language_s']=y
del df['language']

In [ ]:
x=[]
y=[]
z=[]
for i in df['job']:
  j=i.split('/')
  z.append(len(j))
  x.append(j[0])
  if len(j)>1:
    y.append(j[1])
  else:
    y.append('other')
df['job_length']=z
df['job_f']=x
df['job_s']=y

In [ ]:
df.tail()

,user_id,username,age,status,sex,orientation,drinks,drugs,height,job,pets,smokes,new_languages,body_profile,education_level,dropped_out,interests,other_interests,location_preference,job_length,job_f,job_s,word_count,char_count,avg_word,stopwords,word_density,len,polarity,subjectivity,street,country,language_length,language_f,language_s
1996,fffe3100390039003700,1563,24,3,1,2,4,0,69.0,18,14,4,1,2,2.0,0,6,21,1,1,18,14,77,488,5.282051,23,6.256410,488,0.238763,0.420875,40,1,1,0,127
1997,fffe3100390039003800,1104,23,3,0,1,4,0,68.0,16,14,0,1,1,2.0,0,14,0,2,3,16,9,74,431,4.773333,14,5.746667,431,0.256300,0.494737,55,1,1,0,127
1998,fffe3100390039003900,357,28,3,1,2,3,0,71.0,16,11,0,1,1,3.0,0,22,20,0,3,16,9,122,750,5.113821,33,6.097561,750,0.196047,0.385152,47,1,1,0,127
1999,fffe3200300030003000,1626,32,2,1,2,1,0,68.0,12,14,0,2,10,3.0,0,16,5,1,1,12,14,155,945,5.070513,52,6.057692,945,0.174624,0.437728,55,1,2,1,70
2000,fffe3200300030003100,1394,41,3,0,2,4,0,67.0,0,6,0,2,3,2.0,0,15,14,0,3,0,11,10,61,4.727273,1,5.545455,61,0.500000,0.680000,40,1,2,1,109


In [ ]:
s=['username', 'status', 'sex', 'orientation', 'drinks',
       'drugs', 'job', 'pets', 'smokes',
       'new_languages', 'body_profile', 'dropped_out',
        'interests', 'other_interests', 'location_preference','job_f','job_s','street','country','language_f','language_s']
for i in s:
  from sklearn.preprocessing import LabelEncoder
  le = LabelEncoder()
  df[i] = le.fit_transform(df[i])

In [ ]:
test=df.copy()
train=df.copy()

In [ ]:
#Statical Features Extracting from train and test rows after droping Inovicedate or Exctracting features
#Train
train['Std'] = train.std(axis=1).round(2)
train['quantile1'] = train.quantile(q=0.01,axis=1).round(2)
train['quantile3'] = train.quantile(q=0.25,axis=1).round(2)
train['quantile5'] = train.quantile(q=0.50,axis=1).round(2)                                              
train['Mean'] = train.mean(axis=1).round(2)
#Test
test['Std'] = test.std(axis=1).round(2)
test['quantile1'] = test.quantile(q=0.01,axis=1).round(2)
test['quantile3'] = test.quantile(q=0.25,axis=1).round(2)
test['quantile5'] = test.quantile(q=0.50,axis=1).round(2)                                              
test['Mean'] = test.mean(axis=1).round(2)

In [ ]:
del test['user_id']

In [ ]:
train_df=train
test_df=test

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['user_id'] = le.fit_transform(train['user_id'])
le.classes_

array(['fffe3100', 'fffe31003000', 'fffe310030003000', ...,
       'fffe390039003700', 'fffe390039003800', 'fffe390039003900'],
      dtype=object)

In [ ]:
X = train_df.drop(labels=['user_id'], axis=1)
y = train_df['user_id'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.10, random_state=101)

In [ ]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((1800, 39), (1800,), (201, 39), (201,))

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.metrics import log_loss

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'multiclass',
         'num_class': 2001,
         'boosting': 'gbdt',  
         'metric': 'multi_logloss',
         'learning_rate': 0.4, 
         'num_iterations': 400,
         'num_leaves': 31,
         'max_depth': -1,
         'min_data_in_leaf': 30,
         'bagging_fraction':0.9,
         'bagging_freq': 1,
         'feature_fraction': 0.9,
         'lambda_l2': 0.9,
         'min_data_per_group': 500,
         'max_bin': 500,
         'is_unbalance':True
         }

clf = lgb.train(params=param, 
                early_stopping_rounds=200,
                verbose_eval=100,
                train_set=train_data,
                valid_sets=[test_data])

y_pred = clf.predict(X_cv)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[100]	valid_0's multi_logloss: 34.5388
[200]	valid_0's multi_logloss: 34.5388
Early stopping, best iteration is:
[1]	valid_0's multi_logloss: 34.5388


In [ ]:
y_pred_x = clf.predict(test)

In [ ]:
y_pred_class = le.inverse_transform(train['user_id'])

In [ ]:
submission = pd.DataFrame(data=y_pred_x, columns=y_pred_class)
submission=submission
submission.tail()

,fffe3100,fffe3200,fffe3300,fffe3400,fffe3500,fffe3600,fffe3700,fffe3800,fffe3900,fffe31003000,fffe31003100,fffe31003200,fffe31003300,fffe31003400,fffe31003500,fffe31003600,fffe31003700,fffe31003800,fffe31003900,fffe32003000,fffe32003100,fffe32003200,fffe32003300,fffe32003400,fffe32003500,fffe32003600,fffe32003700,fffe32003800,fffe32003900,fffe33003000,fffe33003100,fffe33003200,fffe33003300,fffe33003400,fffe33003500,fffe33003600,fffe33003700,fffe33003800,fffe33003900,fffe34003000,...,fffe3100390036003200,fffe3100390036003300,fffe3100390036003400,fffe3100390036003500,fffe3100390036003600,fffe3100390036003700,fffe3100390036003800,fffe3100390036003900,fffe3100390037003000,fffe3100390037003100,fffe3100390037003200,fffe3100390037003300,fffe3100390037003400,fffe3100390037003500,fffe3100390037003600,fffe3100390037003700,fffe3100390037003800,fffe3100390037003900,fffe3100390038003000,fffe3100390038003100,fffe3100390038003200,fffe3100390038003300,fffe3100390038003400,fffe3100390038003500,fffe3100390038003600,fffe3100390038003700,fffe3100390038003800,fffe3100390038003900,fffe3100390039003000,fffe3100390039003100,fffe3100390039003200,fffe3100390039003300,fffe3100390039003400,fffe3100390039003500,fffe3100390039003600,fffe3100390039003700,fffe3100390039003800,fffe3100390039003900,fffe3200300030003000,fffe3200300030003100
1996,1.120407e-15,1.120407e-15,0.000546,0.000545,0.000546,0.000548,0.000546,1.120407e-15,0.000545,0.000546,0.000622,0.000622,0.000545,1.120407e-15,1.120407e-15,0.000545,0.000622,0.000546,0.000545,0.000546,0.000546,0.000546,0.000545,0.000545,0.000545,0.000546,0.000545,0.000546,0.000545,0.000548,0.000546,0.000545,0.000622,0.000545,0.000546,0.000622,0.000545,1.120407e-15,1.120407e-15,1.120407e-15,...,0.000546,1.120407e-15,0.000546,0.000545,0.000622,0.000546,0.000545,0.000547,1.120407e-15,0.000545,0.000622,0.000545,0.000549,0.000545,0.000546,1.120407e-15,0.000545,0.000545,0.000545,0.000545,1.120407e-15,0.000547,0.000546,0.000546,0.000546,0.000545,0.000622,0.000622,0.000546,0.000545,0.000545,1.120407e-15,0.000546,1.120407e-15,0.000546,0.000546,0.000545,0.000546,0.000545,0.000546
1997,1.111204e-15,1.111204e-15,0.000541,0.000541,0.000541,0.000543,0.000541,1.111204e-15,0.000541,0.000541,0.000617,0.000617,0.000541,1.111204e-15,1.111204e-15,0.000541,0.000617,0.000541,0.000541,0.000541,0.000541,0.000542,0.000541,0.000541,0.000541,0.000541,0.000541,0.000541,0.000541,0.000595,0.000541,0.000541,0.000617,0.000541,0.000542,0.000617,0.000541,1.111204e-15,1.111204e-15,1.111204e-15,...,0.000542,1.111204e-15,0.000542,0.000541,0.000617,0.000541,0.000541,0.000542,1.111204e-15,0.000541,0.000617,0.000541,0.000545,0.000541,0.000541,1.111204e-15,0.000541,0.000541,0.000541,0.000541,1.111204e-15,0.000543,0.000542,0.000542,0.000541,0.000541,0.000617,0.000617,0.000541,0.000541,0.000541,1.111204e-15,0.000541,1.111204e-15,0.000542,0.000541,0.000541,0.000541,0.000541,0.000541
1998,1.123046e-15,1.123046e-15,0.000547,0.000546,0.000547,0.000549,0.000547,1.123046e-15,0.000546,0.000547,0.000624,0.000624,0.000547,1.123046e-15,1.123046e-15,0.000547,0.000624,0.000547,0.000547,0.000547,0.000547,0.000548,0.000546,0.000546,0.000547,0.000547,0.000547,0.000547,0.000547,0.000550,0.000547,0.000546,0.000624,0.000546,0.000547,0.000624,0.000546,1.123046e-15,1.123046e-15,1.123046e-15,...,0.000548,1.123046e-15,0.000547,0.000547,0.000624,0.000547,0.000547,0.000548,1.123046e-15,0.000546,0.000624,0.000547,0.000550,0.000547,0.000547,1.123046e-15,0.000546,0.000547,0.000546,0.000546,1.123046e-15,0.000549,0.000547,0.000547,0.000547,0.000546,0.000624,0.000624,0.000547,0.000547,0.000547,1.123046e-15,0.000547,1.123046e-15,0.000548,0.000547,0.000547,0.000547,0.000546,0.000547
1999,1.114857e-15,1.114857e-15,0.000543,0.000542,0.000543,0.000545,0.000543,1.114857e-15,0.000542,0.000543,0.000619,0.000619,0.000543,1.114857e-15,1.114857e-15,0.000543,0.000619,0.000543,0.000543,0.000543,0.000543,0.000544,0.000542,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000546,0.000543,0.000542,0.0

In [ ]:
submission['user_id']=y_pred_class

In [ ]:
submission.tail()

,fffe3100,fffe3200,fffe3300,fffe3400,fffe3500,fffe3600,fffe3700,fffe3800,fffe3900,fffe31003000,fffe31003100,fffe31003200,fffe31003300,fffe31003400,fffe31003500,fffe31003600,fffe31003700,fffe31003800,fffe31003900,fffe32003000,fffe32003100,fffe32003200,fffe32003300,fffe32003400,fffe32003500,fffe32003600,fffe32003700,fffe32003800,fffe32003900,fffe33003000,fffe33003100,fffe33003200,fffe33003300,fffe33003400,fffe33003500,fffe33003600,fffe33003700,fffe33003800,fffe33003900,fffe34003000,...,fffe3100390036003300,fffe3100390036003400,fffe3100390036003500,fffe3100390036003600,fffe3100390036003700,fffe3100390036003800,fffe3100390036003900,fffe3100390037003000,fffe3100390037003100,fffe3100390037003200,fffe3100390037003300,fffe3100390037003400,fffe3100390037003500,fffe3100390037003600,fffe3100390037003700,fffe3100390037003800,fffe3100390037003900,fffe3100390038003000,fffe3100390038003100,fffe3100390038003200,fffe3100390038003300,fffe3100390038003400,fffe3100390038003500,fffe3100390038003600,fffe3100390038003700,fffe3100390038003800,fffe3100390038003900,fffe3100390039003000,fffe3100390039003100,fffe3100390039003200,fffe3100390039003300,fffe3100390039003400,fffe3100390039003500,fffe3100390039003600,fffe3100390039003700,fffe3100390039003800,fffe3100390039003900,fffe3200300030003000,fffe3200300030003100,user_id
1996,1.117154e-15,1.117154e-15,0.000545,0.000544,0.000545,0.000546,0.000544,1.117154e-15,0.000544,0.000544,0.000621,0.000621,0.000544,1.117154e-15,1.117154e-15,0.000544,0.000621,0.000544,0.000544,0.000544,0.000545,0.000545,0.000544,0.000544,0.000544,0.000544,0.000544,0.000544,0.000544,0.000547,0.000544,0.000544,0.000621,0.000544,0.000544,0.000621,0.000544,1.117154e-15,1.117154e-15,1.117154e-15,...,1.117154e-15,0.000544,0.000544,0.000621,0.000544,0.000544,0.000546,1.117154e-15,0.000545,0.000621,0.000544,0.000548,0.000544,0.000544,1.117154e-15,0.000544,0.000544,0.000544,0.000544,1.117154e-15,0.000546,0.000544,0.000545,0.000544,0.000544,0.000621,0.000621,0.000545,0.000544,0.000544,1.117154e-15,0.000544,1.117154e-15,0.000545,0.000544,0.000544,0.000544,0.000544,0.000544,fffe3100390039003700
1997,1.110846e-15,1.110846e-15,0.000542,0.000541,0.000541,0.000543,0.000541,1.110846e-15,0.000541,0.000541,0.000617,0.000617,0.000541,1.110846e-15,1.110846e-15,0.000541,0.000617,0.000541,0.000541,0.000541,0.000542,0.000542,0.000541,0.000541,0.000541,0.000541,0.000541,0.000541,0.000541,0.000595,0.000541,0.000541,0.000617,0.000541,0.000541,0.000617,0.000541,1.110846e-15,1.110846e-15,1.110846e-15,...,1.110846e-15,0.000541,0.000541,0.000617,0.000541,0.000541,0.000543,1.110846e-15,0.000542,0.000617,0.000541,0.000544,0.000541,0.000541,1.110846e-15,0.000541,0.000541,0.000541,0.000541,1.110846e-15,0.000543,0.000541,0.000541,0.000541,0.000541,0.000617,0.000617,0.000607,0.000541,0.000541,1.110846e-15,0.000541,1.110846e-15,0.000542,0.000541,0.000541,0.000541,0.000541,0.000541,fffe3100390039003800
1998,1.121951e-15,1.121951e-15,0.000547,0.000546,0.000547,0.000548,0.000546,1.121951e-15,0.000546,0.000546,0.000623,0.000623,0.000546,1.121951e-15,1.121951e-15,0.000546,0.000623,0.000546,0.000546,0.000547,0.000547,0.000547,0.000546,0.000546,0.000546,0.000546,0.000546,0.000546,0.000546,0.000549,0.000546,0.000546,0.000623,0.000546,0.000547,0.000623,0.000546,1.121951e-15,1.121951e-15,1.121951e-15,...,1.121951e-15,0.000547,0.000546,0.000623,0.000546,0.000546,0.000549,1.121951e-15,0.000548,0.000623,0.000546,0.000550,0.000546,0.000546,1.121951e-15,0.000546,0.000546,0.000546,0.000546,1.121951e-15,0.000548,0.000547,0.000547,0.000546,0.000546,0.000623,0.000623,0.000547,0.000546,0.000546,1.121951e-15,0.000547,1.121951e-15,0.000548,0.000546,0.000546,0.000547,0.000546,0.000546,fffe3100390039003900
1999,1.113920e-15,1.113920e-15,0.000543,0.000542,0.000543,0.000545,0.000543,1.113920e-15,0.000542,0.000542,0.000619,0.000619,0.000542,1.113920e-15,1.113920e-15,0.000542,0.000619,0.000542,0.000542,0.000543,0.000543,0.000543,0.000542,0.000542,0.000542,0.000542,0.000542,0.000542,0.000542,0.00054

In [ ]:
submission.to_csv('lg_match0.csv', index=False)